In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from ta.momentum import RSIIndicator
from ta.trend import MACD, EMAIndicator, ADXIndicator
from ta.volatility import AverageTrueRange
from ta.trend import PSARIndicator
from datetime import datetime

# Load the historical data
df = pd.read_csv('/content/BTC_2019_2023_1d.csv')  # Replace with your CSV file path
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df = df.dropna(subset=['datetime'])
df.set_index('datetime', inplace=True)

# Ensure required columns exist
required_columns = ['open', 'high', 'low', 'close', 'signals']
for column in required_columns:
    if column not in df.columns:
        df[column] = 0 if column == 'signals' else np.nan

df.fillna(method='ffill', inplace=True)
df['high'].fillna(df['close'], inplace=True)
df['low'].fillna(df['close'], inplace=True)

# Calculate Heikin Ashi Candles
df['HA_close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
df['HA_open'] = (df['open'].shift(1) + df['close'].shift(1)) / 2
df['HA_high'] = df[['high', 'HA_open', 'HA_close']].max(axis=1)
df['HA_low'] = df[['low', 'HA_open', 'HA_close']].min(axis=1)

# Technical Indicators
df['RSI'] = RSIIndicator(close=df['HA_close'], window=14).rsi()
macd = MACD(close=df['HA_close'], window_slow=26, window_fast=12, window_sign=9)
df['MACD'] = macd.macd()
df['MACD_Signal'] = macd.macd_signal()
df['MACD_Hist'] = macd.macd_diff()
df['ema_05'] = EMAIndicator(close=df['HA_close'], window=5).ema_indicator()
df['ema_10'] = EMAIndicator(close=df['HA_close'], window=10).ema_indicator()
df['ema_30'] = EMAIndicator(close=df['HA_close'], window=30).ema_indicator()
df['ATR'] = AverageTrueRange(high=df['HA_high'], low=df['HA_low'], close=df['HA_close'], window=14).average_true_range()
adx = ADXIndicator(high=df['HA_high'], low=df['HA_low'], close=df['HA_close'], window=14)
df['ADX'] = adx.adx()
df['ADX_Pos'] = adx.adx_pos()
df['ADX_Neg'] = adx.adx_neg()
psar = PSARIndicator(high=df['HA_high'], low=df['HA_low'], close=df['HA_close'], step=0.02, max_step=0.2)
df['PSAR'] = psar.psar()

# RandomForest Model for Future Price Prediction
X = df[['open', 'close', 'RSI', 'MACD', 'ema_10', 'ADX']].dropna()
y = (df['close'].shift(-1) > df['close']).astype(int)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X[:-1], y[:-1])
df.loc[X.index, 'Future_Prediction'] = model.predict(X)

# Trading Strategy
rsi_buy_threshold = 30
rsi_sell_threshold = 60
adx_buy_threshold = 25
adx_sell_threshold = 25

in_position = False
position_type = 0
entry_price = 0
net_profit = 0
balance = 10000
btc_amount = 0
trade_count = 0
winning_trades = 0
losing_trades = 0
signal_history = [0] * len(df)
trade_type = [None] * len(df)
balance_history = []

for i, (index, row) in enumerate(df.iterrows()):
    current_price = row['close']
    rsi, macd, macd_signal, adx, psar = row['RSI'], row['MACD'], row['MACD_Signal'], row['ADX'], row['PSAR']
    future_prediction = row['Future_Prediction']

    buy_condition = (
        rsi > rsi_buy_threshold and row['ema_10'] > row['ema_30'] and macd > macd_signal and current_price > row['ema_10']
        and adx > adx_buy_threshold and row['ADX_Pos'] > row['ADX_Neg'] and current_price > psar and future_prediction == 1
    )

    if buy_condition and not in_position:
        entry_price = current_price
        btc_amount = balance / entry_price
        in_position = True
        position_type = 1
        trade_count += 1
        signal_history[i] = 1
        trade_type[i] = 'long'

    sell_condition = (
        current_price <= psar and rsi < rsi_sell_threshold and macd < macd_signal and adx < adx_sell_threshold
    )

    if in_position and position_type == 1 and sell_condition:
        balance = btc_amount * current_price
        profit = (current_price - entry_price) * btc_amount
        net_profit += profit
        winning_trades += 1 if profit > 0 else 0
        losing_trades += 1 if profit < 0 else 0
        in_position = False
        signal_history[i] = -1
        trade_type[i] = 'exit'

    balance_history.append(balance)

df['signals'] = signal_history
df['trade_type'] = trade_type

# Save DataFrame to CSV
output_filename = 'Strategy_with_RandomForest.csv'
df.to_csv(output_filename, index=True)
print(f"File saved as {output_filename}")
